https://github.com/Toblerity/Fiona/issues/944

In [ ]:
import fiona

In [ ]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas

In [ ]:
import maup
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.updaters import Tally, cut_edges, exterior_boundaries, exterior_boundaries_as_a_set
from networkx import is_connected, connected_components


In [ ]:
graph = Graph.from_file("./az_precincts.zip", ignore_errors=True)
precincts = geopandas.read_file("./az_precincts.zip")

In [ ]:
components = list(connected_components(graph))
biggest_component_size = max(len(c) for c in components)
problem_components = [c for c in components if len(c) != biggest_component_size]
for component in problem_components:
    for node in component:
        graph.remove_node(node)
is_connected(graph)

In [ ]:
election = Election("PRES16", {"Dem": "PRES16D", "Rep": "PRES16R"})

initial_partition = GeographicPartition(
    graph,
    assignment="CD",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOTPOP", alias="population"),
        "PRES16": election
    }
)

In [ ]:
from gerrychain.constraints.contiguity import contiguous_components, contiguous
from gerrychain import Partition
bad_nodeview = contiguous_components(initial_partition).get('01')[1].nodes()
bad_nodeview

In [ ]:
graph

In [ ]:
contiguous_components(initial_partition)

Arizona has some missing edges in its graph

In [ ]:
graph.add_edge(208,378)
graph.add_edge(208,202)
graph.add_edge(208,780)
graph.add_edge(208,90)
graph.add_edge(208,42)

In [ ]:
graph

Recreate the initial partition

In [ ]:
contiguous_components(initial_partition)

In [ ]:
election = Election("PRES16", {"Dem": "PRES16D", "Rep": "PRES16R"})

initial_partition = GeographicPartition(
    graph,
    assignment="CD",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOTPOP", alias="population"),
        "PRES16": election
    }
)

In [ ]:
for district, pop in initial_partition["population"].items():
    print("District {}: {}".format(district, pop))

In [ ]:
sum_population = sum(initial_partition["population"].values())
ideal_population = sum_population / len(initial_partition)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=ideal_population,
                   epsilon=.05,
                   node_repeats=2
                  )

In [ ]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, .05)

In [ ]:
def district_diff(partition1, partition2):
    percentage_change = []
    for (district1, graph1), (district2, graph2) in zip(contiguous_components(partition1).items(), contiguous_components(partition2).items()):
        if district1 == district2:
            set1 = set(graph1[0].nodes)
            set2 = set(graph2[0].nodes)
            if set1 != set2:
                set_diff1 = set1 - set2
                set_diff2 = set2 - set1
                set_intersection = set1 & set2
                diff = len(set_intersection)/len(set1)
                if diff > 1:
                    percentage_change.append(0)
                else:
                    percentage_change.append(diff)
            else:
                percentage_change.append(1)
    return percentage_change

In [ ]:
from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous, contiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
steps = 1000
chain = MarkovChain(
    proposal=proposal,
    constraints=[single_flip_contiguous],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=steps
)

In [ ]:
last1 = None
best_partition = None
best_partition_similarity = 1
district_percent_change_per_partition = []
for partition in chain.with_progress_bar():
    district_differences = district_diff(initial_partition, partition)
    district_percent_change_per_partition.append(district_differences)
    last1 = partition
    partition_similarity = np.mean(district_differences)
    if best_partition_similarity > partition_similarity:
        best_partition_similarity = partition_similarity
        best_partition = partition

In [ ]:
import matplotlib.pyplot as plt
y = np.mean(district_percent_change_per_partition, axis=1)
plt.plot(y)

In [ ]:
initial_partition.plot(figsize=(10, 10), cmap="tab20")
plt.axis('off')
plt.show()

In [ ]:
best_partition.plot(figsize=(10, 10), cmap="tab20")
plt.axis('off')
plt.show()

In [ ]:
last1.plot(figsize=(10, 10), cmap="tab20")
plt.axis('off')
plt.show()